---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [51]:

'''Returns a DataFrame of towns and the states they are in from the 
university_towns.txt list. The format of the DataFrame should be:
DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
columns=["State", "RegionName"]  )

The following cleaning needs to be done:

1. For "State", removing characters from "[" to the end.
2. For "RegionName", when applicable, removing every character from " (" to the end.
3. Depending on how you read the data, you may need to remove newline character '\n'. '''
with open('university_towns.txt', 'r') as doc:
    uni_towns = doc.read().splitlines()
    uni_towns = pd.DataFrame(uni_towns, columns= ["State"])
    uni_towns['RegionName']=uni_towns['State'].str.partition('(')[0]
    uni_towns['State'] = uni_towns[uni_towns.State.str.contains("\[edit]")]
    uni_towns['State'] = uni_towns['State'].str.strip('[edit]')
    uni_towns['State'].fillna(method='ffill', inplace=True)
    uni_towns['RegionName'] = uni_towns.RegionName[~uni_towns.RegionName.str.contains("\[edit]") == True]
    uni_towns.dropna(inplace=True)

        #Looping it came out as faster, disapoing
def get_list_of_university_towns():
    return uni_towns

get_list_of_university_towns()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
6,Alabama,Troy
7,Alabama,Tuscaloosa
8,Alabama,Tuskegee
10,Alaska,Fairbanks
12,Arizona,Flagstaff


In [36]:
def get_list_of_university_towns1():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    data = []
    state = None
    state_towns = []
    with open('university_towns.txt') as file:
        for line in file:
            thisLine = line[:-1]
            if thisLine[-6:] == '[edit]':
                state = thisLine[:-6]
                continue
            if '(' in line:
                town = thisLine[:thisLine.index('(')-1]
                state_towns.append([state,town])
            else:
                town = thisLine
                state_towns.append([state,town])
            data.append(thisLine)
    df = pd.DataFrame(state_towns,columns = ['State','RegionName'])
    return df
get_list_of_university_towns1()['RegionName'].values

array(['Auburn', 'Florence', 'Jacksonville', 'Livingston', 'Montevallo',
       'Troy', 'Tuscaloosa', 'Tuskegee', 'Fairbanks', 'Flagstaff', 'Tempe',
       'Tucson', 'Arkadelphia', 'Conway', 'Fayetteville', 'Jonesboro',
       'Magnolia', 'Monticello', 'Russellville', 'Searcy', 'Angwin',
       'Arcata', 'Berkeley', 'Chico', 'Claremont', 'Cotati', 'Davis',
       'Irvine', 'Isla Vista', 'University Park, Los Angeles', 'Merced',
       'Orange', 'Palo Alto', 'Pomona', 'Redlands', 'Riverside',
       'Sacramento', 'University District, San Bernardino', 'San Diego',
       'San Luis Obispo', 'Santa Barbara', 'Santa Cruz', 'Turlock',
       'Westwood, Los Angeles', 'Whittier', 'Alamosa', 'Boulder',
       'Durango', 'Fort Collins', 'Golden', 'Grand Junction', 'Greeley',
       'Gunnison', 'Pueblo, Colorado', 'Fairfield', 'Middletown',
       'New Britain', 'New Haven', 'New London', 'Storrs', 'Willimantic',
       'Dover', 'Newark', 'Ave Maria', 'Boca Raton', 'Coral Gables',
       'DeLand

In [5]:
df_gdp = pd.read_excel('gdplev.xls', skiprows = 7, parse_cols="E,F", names=["Quarter","GDP"])
df_gdp = df_gdp[df_gdp['Quarter'] >='2000q1']  
df_gdp.head()

,Quarter,GDP
212,2000q1,10031.0
213,2000q2,10278.3
214,2000q3,10357.4
215,2000q4,10472.3
216,2001q1,10508.1


In [6]:
# Recesion year search
def recesion_search():
    for year in range(0, len(df_gdp) -4):
          if (df_gdp.iloc[year][1] > df_gdp.iloc[year+1][1]) and (
            df_gdp.iloc[year+1][1] > df_gdp.iloc[year+2][1]) and (
            df_gdp.iloc[year+2][1] < df_gdp.iloc[year+3][1]) and (
            df_gdp.iloc[year+3][1] < df_gdp.iloc[year+4][1]):
            return year
year_index = recesion_search()
print(year_index)

35


In [7]:

def get_recession_start():
    return df_gdp.iloc[year_index][0]

get_recession_start()
#df_gdp['GDP'].head()


'2008q4'

###

In [8]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''

    return df_gdp.iloc[year_index+4][0]

get_recession_end()

'2009q4'

In [9]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''

    return df_gdp.iloc[year_index+2][0]
get_recession_bottom()

'2009q2'

In [32]:
years = list(range(2000,2017))
quarters = ['q1','q2','q3','q4']
quarter_years = []
for i in years:
    for x in quarters:
        quarter_years.append((str(i)+x))
years_named = quarter_years[:67]

In [11]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    data = pd.read_csv('City_Zhvi_AllHomes.csv', )
    data.drop(['RegionID','CountyName','Metro', 'SizeRank'], axis = 1, inplace =1)
    data['State'] = data['State'].map(states)
    data.set_index(['RegionName','State'], inplace =True)


    columns = pd.Series(data.columns)
    columns_left = columns[columns >= '2000-01']
    columns_to_delete = columns[columns <'2000-01']
    
    data.drop(columns_to_delete, inplace=True, axis=1)
    
    columns_in_quaters = [list(data.columns)[x:x+3] for x in range(0, len(list(data.columns)), 3)]
    
    # new columns
    for col,quarter in zip(years_named,columns_in_quaters):
        data[col] = data[quarter].mean(axis=1)
    
    data = data[years_named]
    return data
convert_housing_data_to_quarters().head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
RegionName,State,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
Los Angeles,California,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Chicago,Illinois,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Philadelphia,Pennsylvania,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Phoenix,Arizona,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [12]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # THERE ARE NO UNIVERSITY TOWNS(university_towns.txt ) IN HOUSING DATA ('City_Zhvi_AllHomes.csv'), TEST CANNOT BE DONE PROPERLY
    
    
    return "ANSWER"

In [43]:

def function():
    unitowns = get_list_of_university_towns()
    unitowns=unitowns.reindex(columns=['RegionName','State'])
    bottom = get_recession_bottom()
    start = get_recession_start()
    housing_data = convert_housing_data_to_quarters()
    bstart = housing_data.columns[housing_data.columns.get_loc(start)]

    housing_data['ratio'] = (housing_data[bottom] - housing_data[bstart])/housing_data[bottom]
    uni_town = get_list_of_university_towns()['RegionName']
    unitowns_hdata = pd.merge(housing_data, uni_town,how='inner',on=['State','RegionName'])
    uni_town = set(uni_town)
    uni_town1 = set(['Los Angeles', "San Jose"])
    housing_data.reset_index(inplace=True)
    
    def is_uni_town(row):
        if row['RegionName'] in uni_town:
            return True
        else:
            return False

    housing_data['IsUniTown'] = housing_data.apply(is_uni_town, axis=1)
    
    no_uni = housing_data[housing_data['IsUniTown']==False]['ratio']
    is_uni  = housing_data[housing_data['IsUniTown']==True]['ratio']

    t,p = ttest_ind(no_uni, is_uni)
    better = "non-university town" if is_uni.mean() < no_uni.mean() else "university town"
    
    
    # THERE ARE NO UNIVERSITY TOWNS(university_towns.txt ) IN HOUSING DATA ('City_Zhvi_AllHomes.csv'), TEST CANNOT BE DONE PROPERLY
    
    return  
function()



ValueError: can not merge DataFrame with instance of type <class 'pandas.core.series.Series'>

,RegionName,State,2009q2,2008q3,ratio,uni
